# Финальная работа

In [59]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import warnings
import pickle
import dill
import re
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import (
                            train_test_split,
                            cross_val_score,
                            KFold)
from sklearn.ensemble import(
                     GradientBoostingClassifier,
                     RandomForestClassifier)
from sklearn.preprocessing import(
                             StandardScaler,
                             OneHotEncoder)
from sklearn.metrics import (
                            roc_auc_score,
                            classification_report)


In [2]:
# оформление — скрытие предупреждений
warnings.filterwarnings("ignore")

# константы
RS = 42 # RANDOM_STATE

In [3]:
# Функции
def get_info(data):
    '''
    Возвращает основную информацию о датасете
    '''
    print(f'Основные показатели датасета:')
    print('-'*40)
    print(f'Количество признаков: {data.shape[1]}')
    print('-'*40)
    print(f'Количество сэмплов: {data.shape[0]}')
    print('-'*40,end='\n\n')
    print(f'Количество дубликатов: {data.duplicated().sum()}')
    print('-'*40,end='\n\n')
    print('Пример данных:')
    display(data.head(3))


def calculate_missing_data(data):

    # Вывод основной информации о DataFrame
    data.info()

    # Расчет пропусков и процентов пропусков
    missing_data = data.isna().sum()
    missing_percent = (missing_data / len(data)) * 100
    missing_percent = missing_percent.round(2)

    missing_info = pd.DataFrame({
        'Пропуски': missing_data,
        'Процент пропусков': missing_percent
    })

    return missing_info


def check_missing_values(data):
    '''
    Проверка пропусков в признаках
    '''
    for column in data.columns:
        missing_values_count = data[column].isna().sum()
        print(f"Пропущенных значений в признаке {column}: {missing_values_count}")

### Знакомство с данными

In [4]:
df1 = pd.read_csv('ga_sessions.csv', low_memory=False)

In [5]:
df = pd.read_csv('ga_hits.csv')

In [6]:
# Получаем основную информацию о датасете df1
get_info(df1)

Основные показатели датасета:
----------------------------------------
Количество признаков: 18
----------------------------------------
Количество сэмплов: 1860042
----------------------------------------

Количество дубликатов: 0
----------------------------------------

Пример данных:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk


In [7]:
# Получаем основную информацию о датасете df
get_info(df)

Основные показатели датасета:
----------------------------------------
Количество признаков: 11
----------------------------------------
Количество сэмплов: 15726470
----------------------------------------

Количество дубликатов: 0
----------------------------------------

Пример данных:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN


## Оценка полноты и чистоты данных

In [8]:
calculate_missing_data(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 18 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              int64 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
dtypes: int64(1), object(17)
memory usage: 255.4+ MB


,Пропуски,Процент пропусков
session_id,0,0.00
client_id,0,0.00
visit_date,0,0.00
visit_time,0,0.00
visit_number,0,0.00
utm_source,97,0.01
utm_medium,0,0.00
utm_campaign,219603,11.81
utm_adcontent,335615,18.04
utm_keyword,1082061,58.17


In [9]:
calculate_missing_data(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_type        object 
 5   hit_referer     object 
 6   hit_page_path   object 
 7   event_category  object 
 8   event_action    object 
 9   event_label     object 
 10  event_value     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.3+ GB


,Пропуски,Процент пропусков
session_id,0,0.00
hit_date,0,0.00
hit_time,9160322,58.25
hit_number,0,0.00
hit_type,0,0.00
hit_referer,6274804,39.90
hit_page_path,0,0.00
event_category,0,0.00
event_action,0,0.00
event_label,3760184,23.91


In [10]:
# Удаляем неинформативные признаки, а также признаки, где процент пропусков 99-100%
columns_for_drop = ['device_model', 'utm_keyword', 'device_screen_resolution', 'client_id', 
                    'utm_campaign', 'utm_adcontent', 'visit_date', 'visit_time']
df1 = df1.drop(columns=columns_for_drop) 
df1.head()

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,NaN,Xiaomi,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,NaN,Xiaomi,Chrome,Russia,Moscow


In [11]:
columns_for_drop1 = ['event_value', 'hit_time', 'hit_referer', 'event_label']
df = df.drop(columns=columns_for_drop1) 
df.head()

,session_id,hit_date,hit_number,hit_type,hit_page_path,event_category,event_action
0,5639623078712724064.1640254056.1640254056,2021-12-23,30,event,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show
1,7750352294969115059.1640271109.1640271109,2021-12-23,41,event,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show
2,885342191847998240.1640235807.1640235807,2021-12-23,49,event,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show
3,142526202120934167.1640211014.1640211014,2021-12-23,46,event,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show
4,3450086108837475701.1640265078.1640265078,2021-12-23,79,event,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show


#  Разведочный анализ данных

In [12]:
# Выделяем марку автомобиля из признака
def extract_short_model(hit_page_path):
    '''
    Выделяет марку автомобиля из признака
    '''
    parts = hit_page_path.split('/')
    if len(parts) > 3:
        short_model = parts[3]
        # Удаляем параметры из строки и оставляем только буквенно-цифровые символы и дефисы
        short_model = re.split(r'[?&]', short_model)[0]
        # Проверяем, что строка состоит только из буквенно-цифровых символов и дефисов
        if re.match(r'^[a-zA-Z0-9\-]+$', short_model):
            return short_model
    return 'unknown'

In [13]:
# Создаём новую фичу
df['short_model'] = df['hit_page_path'].apply(extract_short_model)

In [14]:
# Определяем целевое действие
df.event_action = df.event_action.replace({'sub_car_claim_click' : 1, 'sub_car_claim_submit_click': 1,
'sub_open_dialog_click': 1, 'sub_custom_question_submit_click': 1,
'sub_call_number_click': 1, 'sub_callback_submit_click': 1, 'sub_submit_success': 1,
'sub_car_request_submit_click' : 1})

In [15]:
df.loc[(df.event_action != 1), 'event_action'] = 0 
df.event_action.value_counts(dropna=False)

event_action
0    15621562
1      104908
Name: count, dtype: int64

In [16]:
# удаляем неинформативные признаки, а также фичи, которые могут считаться утечкой
columns_for_drop2 = ['hit_date', 'hit_number', 'hit_type', 'hit_page_path']
df = df.drop(columns=columns_for_drop2) 
df.head()

,session_id,event_category,event_action,short_model
0,5639623078712724064.1640254056.1640254056,quiz,0,unknown
1,7750352294969115059.1640271109.1640271109,quiz,0,unknown
2,885342191847998240.1640235807.1640235807,quiz,0,volkswagen
3,142526202120934167.1640211014.1640211014,quiz,0,unknown
4,3450086108837475701.1640265078.1640265078,quiz,0,mercedes-benz


#### Обьединение двух датафреймов

##### Проверяем наличие дубликатов в признаке, по которому будем объединять датафреймы

In [17]:
print(df1['session_id'].duplicated().sum())

0


In [18]:
print(df['session_id'].duplicated().sum())

13991860


In [19]:
# Удаляем дубликаты перед объединением
df = df.drop_duplicates(subset=['session_id'])

In [20]:
print(df['session_id'].duplicated().sum())

0


In [21]:
data = pd.merge(left=df1, right=df, on='session_id', how='inner')
get_info(data)

Основные показатели датасета:
----------------------------------------
Количество признаков: 13
----------------------------------------
Количество сэмплов: 1732266
----------------------------------------

Количество дубликатов: 0
----------------------------------------

Пример данных:


,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,event_category,event_action,short_model
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Zlatoust,sub_page_view,0,unknown
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,sub_page_view,0,unknown
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,search_form,0,unknown


#### Исследуем целевой признак

In [22]:
# Удаление пропусков в целевом признаке
data.dropna(subset='event_action', axis=0, inplace=True)

#проверяем баланс значений в признаке
print(data['event_action'].value_counts())
print('\nКоличество пропусков в признаке', data['event_action'].isna().sum())
count_1 = data['event_action'].sum()
count_0 = len(data) - count_1
percent_1 = (count_1 / len(data)) * 100
percent_0 = (count_0 / len(data)) * 100
print(f'\nПроцент значений 1: {percent_1:.2f}%\nПроцент значений 0: {percent_0:.2f}%')

# Исправление типа данных
data['event_action'] = data['event_action'].astype('int64')

event_action
0    1727037
1       5229
Name: count, dtype: int64

Количество пропусков в признаке 0

Процент значений 1: 0.30%
Процент значений 0: 99.70%


#### Вывод: в целевом признаке имеется сильный дисбаланс классов

### Корреляционный анализ

In [25]:
# Изучим корреляцию между числовыми признаками
data['event_action'].corr(data['visit_number'])

0.03173104530039895

In [26]:
# Частотный анализ для категориальных данных
for column in data.select_dtypes(include='object').columns:
    print(f"\nРаспределение значений в признаке '{column}':")
    print(data[column].value_counts())


Распределение значений в признаке 'session_id':
session_id
9055434745589932991.1637753792.1637753792    1
629529258169981545.1626332167.1626332167     1
6295352713792576176.1634065074.1634065074    1
6295350622136026841.1626588901.1626588901    1
6295349887709233242.1639202906.1639202906    1
                                            ..
3524000133754500026.1625142208.1625142208    1
3523993785803856201.1636161866.1636161866    1
352398731068897298.1621686290.1621686290     1
3523985479341028705.1640084833.1640084833    1
9055430416266113553.1640968742.1640968742    1
Name: count, Length: 1732266, dtype: int64

Распределение значений в признаке 'utm_source':
utm_source
ZpYIoDJMcFzVoPFsHGJL    552555
fDLlAcSmythWSCVMvqvL    277060
kjsLglQLzykiRbcDiGcD    245178
MvfHsxITijuriZxsqZqt    175831
BHcvLfOaCWvWTykYqHVe    110963
                         ...  
DWvtKQncdpXXfLBjBmGj         1
ZsHOHNXkbhgIDlKNiFMf         1
hYVrCkhCPSqKBhZYhWVq         1
OboZzsWwJIeGPLeiLGMq         1
sbJRYgVfvc

#### Проверка пропусков после обьединения

In [23]:
check_missing_values(data)

Пропущенных значений в признаке session_id: 0
Пропущенных значений в признаке visit_number: 0
Пропущенных значений в признаке utm_source: 76
Пропущенных значений в признаке utm_medium: 0
Пропущенных значений в признаке device_category: 0
Пропущенных значений в признаке device_os: 1013964
Пропущенных значений в признаке device_brand: 347196
Пропущенных значений в признаке device_browser: 0
Пропущенных значений в признаке geo_country: 0
Пропущенных значений в признаке geo_city: 0
Пропущенных значений в признаке event_category: 0
Пропущенных значений в признаке event_action: 0
Пропущенных значений в признаке short_model: 0


In [24]:
# Удаление строк с пропущенными значениями в указанных столбцах
data.dropna(subset=['device_os', 'device_brand', 'utm_source'], inplace=True)

In [25]:
data.device_os.unique()

array(['Android', 'iOS', '(not set)', 'BlackBerry', 'Tizen', 'Firefox OS',
       'Nokia', 'Samsung', 'Windows Phone'], dtype=object)

In [26]:
data.device_brand.unique()

array(['Huawei', 'Samsung', 'Lenovo', 'Apple', 'Xiaomi', 'Meizu',
       'OnePlus', 'Realme', 'OPPO', '(not set)', 'Philips', 'Vivo',
       'Nokia', 'Alcatel', 'LG', 'BQ', 'Tecno', 'Asus', 'itel', 'Infinix',
       'ZTE', 'Wiko', 'Google', 'Sony', 'Blackview', 'Cubot', 'DOOGEE',
       'DEXP', 'Motorola', 'TP-Link', 'Hisense', 'Acer', 'Oukitel',
       'LeEco', 'Prestigio', 'POCO', 'Vsmart', 'HTC', 'Ulefone', 'CAT',
       'Leagoo', 'InFocus', 'Inoi', 'BlackBerry', 'Micromax', 'Umidigi',
       'Jiake', 'ZOJI', 'Mozilla', 'Neffos', 'Highscreen', 'Karbonn',
       'TCL', 'BLU', 'Haier', 'Vertex', 'Coolpad', 'HOMTOM', 'LeTV', 'A1',
       'Sharp', 'General Mobile', 'Gome', 'Egreat', 'Mito', 'Wileyfox',
       'SenseIT', 'Archos', 'Keecoo', 'Vernee', 'Panasonic', 'InnJoo',
       'Iris', 'Black Fox', 'Lava', 'myPhone', 'AGM', 'Nuu', 'UGOOS',
       'Alldocube', 'MTC', 'Symphony', 'Wigor', 'Oysters', 'Komu', 'Fly',
       'Gionee', 'Artel', 'Ananda', 'Smartisan', 'Elephone', 'Kingplay',
 

In [27]:
data.utm_medium.unique()

array(['banner', 'cpm', 'cpc', 'referral', '(none)', 'organic', 'smm',
       'blogger_channel', 'app', 'push', 'partner', 'cpa', 'email',
       'smartbanner', 'info_text', 'clicks', 'landing', 'blogger_stories',
       'post', 'tg', '(not set)', 'google_cpc', 'outlook',
       'blogger_header', 'vk_smm', 'cpv', 'fb_smm', 'sms',
       'landing_interests', 'yandex_cpc', 'static', 'nkp', 'main_polka',
       'ok_smm', 'stories', 'users_msk'], dtype=object)

In [28]:
# Подсчет количества строк, содержащих "none", "(not set), "unknown""
print("Количество строк, содержащих 'none', '(not set)', 'unknown':")
print((data == '(none)').sum())
print((data == '(not set)').sum())
print((data == 'unknown').sum())

Количество строк, содержащих 'none', '(not set)', 'unknown':
session_id             0
visit_number           0
utm_source             0
utm_medium         41753
device_category        0
device_os              0
device_brand           0
device_browser         0
geo_country            0
geo_city               0
event_category         0
event_action           0
short_model            0
dtype: int64
session_id             0
visit_number           0
utm_source             0
utm_medium           100
device_category        0
device_os              7
device_brand        9018
device_browser         0
geo_country          196
geo_city           17911
event_category         0
event_action           0
short_model            0
dtype: int64
session_id              0
visit_number            0
utm_source              0
utm_medium              0
device_category         0
device_os               0
device_brand            0
device_browser          0
geo_country             0
geo_city                0
eve

In [29]:
# Удаление строк, содержащих "(none)", "(not set)", "unknown" в любом признаке
data = data[~data.apply(lambda row: row.astype(str).str.contains('none|not set|unknown').any(), axis=1)]


In [30]:
print((data == '(none)').sum())
print((data == '(not set)').sum())
print((data == 'unknown').sum())

session_id         0
visit_number       0
utm_source         0
utm_medium         0
device_category    0
device_os          0
device_brand       0
device_browser     0
geo_country        0
geo_city           0
event_category     0
event_action       0
short_model        0
dtype: int64
session_id         0
visit_number       0
utm_source         0
utm_medium         0
device_category    0
device_os          0
device_brand       0
device_browser     0
geo_country        0
geo_city           0
event_category     0
event_action       0
short_model        0
dtype: int64
session_id         0
visit_number       0
utm_source         0
utm_medium         0
device_category    0
device_os          0
device_brand       0
device_browser     0
geo_country        0
geo_city           0
event_category     0
event_action       0
short_model        0
dtype: int64


In [31]:
unique_values = data['device_browser'].unique()
for value in unique_values:
    print(value)

YaBrowser
Android Webview
Chrome
Safari
Samsung Internet
Safari (in-app)
Opera
Android Runtime
UC Browser
Puffin
Instagram 216.0.0.12.135
Amazon Silk
Android Browser
Firefox
NokiaX2-02
Nokia501


In [32]:
# Заменяем содержимое на 'Instagram', если строка содержит это слово
data['device_browser'] = data['device_browser'].apply(lambda x: 'Instagram' if 'Instagram' in x else x)

In [33]:
print(data['device_browser'].unique())

['YaBrowser' 'Android Webview' 'Chrome' 'Safari' 'Samsung Internet'
 'Safari (in-app)' 'Opera' 'Android Runtime' 'UC Browser' 'Puffin'
 'Instagram' 'Amazon Silk' 'Android Browser' 'Firefox' 'NokiaX2-02'
 'Nokia501']


In [34]:
data.short_model.unique()

array(['skoda', 'mercedes-benz', 'haval', 'kia', 'volkswagen', 'volvo',
       'land-rover', 'renault', 'lada-vaz', 'nissan', 'toyota', 'bmw',
       'lexus', 'honda', 'audi', 'porsche', 'mini', 'pajero', 'x3',
       'peugeot', 'sportage', 'granta',
       '3emjrhglwbrbpw76upwd8txntyo1c6ngmv8j7vrqv5ci8ifmtyyausuownlp2upx4mttf7pjgk',
       'hyundai', 'camry', '5-serii', 'rio', 'sorento',
       'lbgkrbejhmv1pokzbxgdge84n', 'outlander',
       'kty9xbrney93n5i9esj3isfhtoau6z641tekgcbj6jy', 'rav-4', 'solaris',
       'wish', 'c-klasse', 'a6', '6', 'atlas', 'o', '308', 'e-klasse',
       'land-cruiser', 'creta', 'a6-allroad', 'corolla', 'priora',
       '2121-4x4', 'lifan', 'chery', 'x5', 'duster', '1111-oka',
       'iir8kfq9avye24oisc', 'geely', 'mazda', 'alphard', 'seltos', '3',
       'hswjwjo63jfn5id8ul613dxtsbebclg12oladbybkqsm8h', 'almera',
       'cruze', 'cx-5', 'note', 'chevrolet', 'h-1', 'i40', '100',
       'lacetti', 'fortuner', 'land-cruiser-prado',
       '6l6jwrd5klmwzuzn

In [35]:
# Список разрешенных значений (читабельных марок и моделей автомобилей)
allowed_models = [
    'skoda', 'mercedes-benz', 'haval', 'kia', 'volkswagen', 'volvo',
    'land-rover', 'renault', 'lada-vaz', 'nissan', 'toyota', 'bmw',
    'lexus', 'honda', 'audi', 'porsche', 'mini', 'pajero', 'x3',
    'peugeot', 'sportage', 'granta', 'hyundai', 'camry', '5-serii', 'rio',
    'sorento', 'outlander', 'rav-4', 'solaris', 'wish', 'c-klasse', 'a6',
    '6', 'atlas', '308', 'e-klasse', 'land-cruiser', 'creta', 'a6-allroad',
    'corolla', 'priora', '2121-4x4', 'lifan', 'chery', 'x5', 'duster',
    '1111-oka', 'geely', 'mazda', 'alphard', 'seltos', '3', 'almera',
    'cruze', 'cx-5', 'note', 'chevrolet', 'h-1', 'i40', '100', 'lacetti',
    'fortuner', 'land-cruiser-prado', 'cerato', 'pathfinder', 'k5',
    'cayenne', 'meriva', 'a5', 'sonata', 'jolion', 'logan', 'tucson',
    'subaru', 'range-rover', 'captur', 'maxima', 'x-trail', 'accent',
    'vesta', 'gls-klasse', 'murano', 'xc90', 'qashqai', 'nx', 'pajero-sport',
    'polo', 'prius', 'x7', 'x6', 'equus', 'gla-klasse-amg',
    'e-klasse-amg', 'xray', 'gt-r', '2115', 'auris', '3-serii', 'lancer',
    'acura', 'santa-fe', 'gs', 'karoq', 'ford', 'infiniti', 'mitsubishi',
    '4-serii', 'opel', 'daewoo', 'grand-vitara', 'i3', 'jaguar', 'octavia',
    '62', 'hilux', 'partner', 'niva', 'kalina', 'ravon', 'estima',
    'challenger', 'carnival', 'uaz', 'm8', 'superb', 'largus', 'nexia',
    'teana', 'amarok', 'm', 'rx', 'insignia', 'terrano', 'elantra',
    'cr-v', 'traveller', 'carina', 'malibu', 'arkana', 'q7', 'patrol',
    'scirocco', 'juke', 'passat', 'asx', 'mondeo', 'corolla-spacio',
    'grand-cherokee', 'lx'
]

In [36]:
# Фильтрация строк, оставляя только те, которые содержат значения
data = data[data['short_model'].isin(allowed_models)]

In [37]:
# Функция для замены символов в строках
def clean_string(s):
    if isinstance(s, str):
        return (s.replace(' ', '_')
                 .replace(',', '')
                 .replace('(', '')
                 .replace(')', '')
                 .replace('/', '')
                 .replace('-', '_')
                 .replace('\\', '_')
                 .replace('.', '_'))
    return s

In [38]:
# Применение функции ко всем элементам DataFrame
data = data.applymap(clean_string)

In [39]:
data['short_model'].unique()

array(['skoda', 'mercedes_benz', 'haval', 'kia', 'volkswagen', 'volvo',
       'land_rover', 'renault', 'lada_vaz', 'nissan', 'toyota', 'bmw',
       'lexus', 'honda', 'audi', 'porsche', 'mini', 'pajero', 'x3',
       'peugeot', 'sportage', 'granta', 'hyundai', 'camry', '5_serii',
       'rio', 'sorento', 'outlander', 'rav_4', 'solaris', 'wish',
       'c_klasse', 'a6', '6', 'atlas', '308', 'e_klasse', 'land_cruiser',
       'creta', 'a6_allroad', 'corolla', 'priora', '2121_4x4', 'lifan',
       'chery', 'x5', 'duster', '1111_oka', 'geely', 'mazda', 'alphard',
       'seltos', '3', 'almera', 'cruze', 'cx_5', 'note', 'chevrolet',
       'h_1', 'i40', '100', 'lacetti', 'fortuner', 'land_cruiser_prado',
       'cerato', 'pathfinder', 'k5', 'cayenne', 'meriva', 'a5', 'sonata',
       'jolion', 'logan', 'tucson', 'subaru', 'range_rover', 'captur',
       'maxima', 'x_trail', 'accent', 'vesta', 'gls_klasse', 'murano',
       'xc90', 'qashqai', 'nx', 'pajero_sport', 'polo', 'prius', 'x7',
   

In [40]:
data.device_browser.unique()

array(['YaBrowser', 'Android_Webview', 'Chrome', 'Safari',
       'Samsung_Internet', 'Safari_in_app', 'Opera', 'Android_Runtime',
       'UC_Browser', 'Puffin', 'Instagram', 'Amazon_Silk',
       'Android_Browser', 'Firefox', 'NokiaX2_02', 'Nokia501'],
      dtype=object)

In [41]:
data.device_os.unique()

array(['Android', 'iOS', 'BlackBerry', 'Nokia', 'Windows_Phone'],
      dtype=object)

In [42]:
# Проверяем количество пропусков в датасете
data.isna().sum().sum()

0

#### Признак `session_id` для построения модели нам не понадобится, удаляем его

In [43]:
# Удаление session_id
data.drop(columns=['session_id'], inplace=True)
data.head()

,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,event_category,event_action,short_model
6,1,TxKUcPpthBDPieTGmVhx,cpc,tablet,Android,Lenovo,YaBrowser,Russia,Saint_Petersburg,card_web,0,skoda
13,3,gVRrcxiDQubJiljoTbGm,referral,mobile,Android,Samsung,Android_Webview,Russia,Sochi,card_web,0,mercedes_benz
31,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Xiaomi,Chrome,Russia,Saint_Petersburg,card_web,0,haval
40,6,kjsLglQLzykiRbcDiGcD,cpc,mobile,iOS,Apple,Safari,Russia,Moscow,card_web,0,mercedes_benz
59,3,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Xiaomi,Chrome,Russia,Tula,card_web,0,kia


#### Кодируем признаки

In [44]:
# Разделение категорий
cat_cols = data.select_dtypes(include='object').columns.to_list()
num_cols = ['visit_number']

In [45]:
cat_cols

['utm_source',
 'utm_medium',
 'device_category',
 'device_os',
 'device_brand',
 'device_browser',
 'geo_country',
 'geo_city',
 'event_category',
 'short_model']

In [46]:
num_cols

['visit_number']

In [47]:
 # кодируем категориальные переменные
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse_output=False,
                    drop='first')
ohe

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [48]:
ohe.fit(data[cat_cols])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [49]:
ohe_data = ohe.transform(data[cat_cols])
ohe_data.shape

(109490, 1266)

In [50]:
 # Выведим новые наименования признаков
feature_names = ohe.get_feature_names_out()
feature_names

array(['utm_source_ArbfvYgWhqxkzywKqpQf',
       'utm_source_BAZCuyHZnaPrMGOMrcCQ',
       'utm_source_BHcvLfOaCWvWTykYqHVe', ..., 'short_model_x_trail',
       'short_model_xc90', 'short_model_xray'], dtype=object)

In [51]:
# Добавляем в исходный датафрейм получившиеся новые признаки
data[feature_names] = ohe_data
print(data.shape)

(109490, 1278)


In [52]:
# нормализация числовых признаков
std_scaler = StandardScaler()
std_scaler

StandardScaler()

In [53]:
std_scaler.fit(data[num_cols])

StandardScaler()

In [54]:
std_scaler_data = std_scaler.transform(data[num_cols])
std_scaler_data.shape

(109490, 1)

In [55]:
# Создаём наименования новых признаков
new_feature_names = [f"{col}_std" for col in num_cols]
new_feature_names

['visit_number_std']

In [56]:
# Добавляем новые признаки в исходный датафрейм
data[new_feature_names] = std_scaler_data
data.shape

(109490, 1279)

In [57]:
# удаляем неинформативные колонки, либо колонки с утечкой данных
columns_for_drop = ['visit_number', 'utm_source','utm_medium',
                    'device_category','device_os','device_brand',
                    'device_browser','geo_country','geo_city',
                    'event_category','short_model']

In [58]:
data_prepared = data.drop(columns=columns_for_drop)
data_prepared.shape

(109490, 1268)

### Подготовка признаков

In [64]:
# Разделение признаков
target = data_prepared['event_action']
features = data_prepared.drop('event_action', axis=1)

In [65]:
# Разделение выборок
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.3,
                                                    stratify=target,
                                                    shuffle=True,
                                                    random_state=RS)
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(76643, 1267)
(32847, 1267)
(76643,)
(32847,)


In [66]:
# Сохраняем имена столбцов
column_names = X_train.columns.tolist()
column_names

['utm_source_ArbfvYgWhqxkzywKqpQf',
 'utm_source_BAZCuyHZnaPrMGOMrcCQ',
 'utm_source_BHcvLfOaCWvWTykYqHVe',
 'utm_source_BKeImrJuRDZcHiSSTdzm',
 'utm_source_CFeqZLBNQdYHxJrTOHjY',
 'utm_source_CXgqTLNTvvxWUWoOfjNF',
 'utm_source_DlnuGwaJBHGNEKdWfOpe',
 'utm_source_DnEUulZAecfGPvdtZBYS',
 'utm_source_FTAuYVNoYYxgvKMpKSLW',
 'utm_source_GmILPdZyuAVJCPsUBHeN',
 'utm_source_GpAkIXsclxDGyILfNlrR',
 'utm_source_HFaOtpcChAlcMuxEAlpu',
 'utm_source_HVQdipMDpgJKXAzecwqn',
 'utm_source_HbolMJUevblAbkHClEQa',
 'utm_source_IRGUHqwEMepMjgCYBVRn',
 'utm_source_ISrKoXQCxqqYvAZICvjs',
 'utm_source_IZEXUFLARCUMynmHNBGo',
 'utm_source_InLrxElufkSAvhfyFcOA',
 'utm_source_KgicpPxiEQfzPlPwQZJq',
 'utm_source_LkGnzVRewoaOHnMCwadT',
 'utm_source_LlBOVIARRTjfgnQNjJre',
 'utm_source_MQvSjpHGoGjbcBjfnLLm',
 'utm_source_MvfHsxITijuriZxsqZqt',
 'utm_source_NGNkCWwKgYFmiCCeZVxg',
 'utm_source_NwLFDlNWnYxuLZEAZppl',
 'utm_source_QxAxdyPLuQMEcrdZWdWb',
 'utm_source_QzPMrfYhYSLYYPtPaBxI',
 'utm_source_RAeIjhqmnmNXpLa

#### Для тренировочной выборки применяем оверсэмплинг для балансировки классов. Для этого используем RandomOverSampler, который увеличивает количество образцов в недопредставленных классах путем случайного дублирования существующих образцов до достижения равномерного распределения классов.

In [67]:
print('Распределение классов до оверсэмплинга:', y_train.value_counts())
ros = RandomOverSampler(sampling_strategy='not majority', random_state=RS)
X_train2, y_train2 = ros.fit_resample(X_train, y_train)
print('Распределение классов после оверсэмплинга:', y_train2.value_counts())

Распределение классов до оверсэмплинга: event_action
0    76255
1      388
Name: count, dtype: int64
Распределение классов после оверсэмплинга: event_action
0    76255
1    76255
Name: count, dtype: int64


## Моделирование

In [69]:
# Задаём эталон в виде оценки dummy классификатора со стратегией стратификации и оценкой на тесте.  
dummy = DummyClassifier(strategy='stratified', random_state=RS)
dummy.fit(X_train2, y_train2)
y_hat = dummy.predict(X_test)
print(roc_auc_score(y_test,y_hat))

0.45479632799426956


In [70]:
# Настройка PCA для уменьшения размерности
pca = PCA(n_components=0.95)  # Оставляем 95% дисперсии

#### Обучение моделей

In [72]:
%%time
mlp = MLPClassifier(random_state=RS)

pipeline = ImbPipeline([
    ('pca', pca),
    ('oversampler', ros),
    ('classifier', mlp)
])

kf = KFold(n_splits=5, shuffle=True, random_state=RS)

scores_mlp = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='roc_auc')

print("ROC-AUC для каждого фолда: ", scores_mlp)
print("Средняя оценка ROC-AUC: ", scores_mlp.mean())

ROC-AUC для каждого фолда:  [0.96650292 0.95670457 0.9739339  0.9741316  0.98925151]
Средняя оценка ROC-AUC:  0.9721048997275288
CPU times: total: 12min 17s
Wall time: 6min 54s


In [73]:
%%time
gb = GradientBoostingClassifier(random_state=RS,
                                n_estimators=300,
                                min_samples_leaf=50,
                                max_depth=5)

pipeline1 = ImbPipeline([
    ('pca', pca),
    ('oversampler', ros),
    ('classifier', gb)
])

kf = KFold(n_splits=5, shuffle=True, random_state=RS)

scores_gb = cross_val_score(pipeline1, X_train, y_train, cv=kf, scoring='roc_auc')

print("ROC-AUC для каждого фолда: ", scores_gb)
print("Средняя оценка ROC-AUC: ", scores_gb.mean())

ROC-AUC для каждого фолда:  [0.99707354 0.98807872 0.99901976 0.99884651 0.99901134]
Средняя оценка ROC-AUC:  0.9964059721816747
CPU times: total: 31min 14s
Wall time: 1h 17min 5s


In [74]:
%%time
rf = RandomForestClassifier(random_state=RS,
                            max_features='log2', 
                            min_samples_leaf=1, 
                            n_estimators=300)

pipeline2 = ImbPipeline([
    ('pca', pca),
    ('oversampler', ros),
    ('classifier', rf)
])

kf = KFold(n_splits=5, shuffle=True, random_state=RS)

scores_rf = cross_val_score(pipeline2, X_train, y_train, cv=kf, scoring='roc_auc')

print("ROC-AUC для каждого фолда: ", scores_rf)
print("Средняя оценка ROC-AUC: ", scores_rf.mean())

ROC-AUC для каждого фолда:  [0.99862764 0.99818089 0.99865703 0.99910024 0.99820276]
Средняя оценка ROC-AUC:  0.9985537087831677
CPU times: total: 13min 55s
Wall time: 7min 5s


In [75]:
%%time
lr = LogisticRegression(random_state=RS,
                        penalty = None, 
                        solver = 'lbfgs', 
                        max_iter = 150)

pipeline3 = ImbPipeline([
    ('pca', pca),
    ('oversampler', ros),
    ('classifier', lr)
])

kf = KFold(n_splits=5, shuffle=True, random_state=RS)

scores_lr = cross_val_score(pipeline3, X_train, y_train, cv=kf, scoring='roc_auc')

print("ROC-AUC для каждого фолда: ", scores_lr)
print("Средняя оценка ROC-AUC: ", scores_lr.mean())

ROC-AUC для каждого фолда:  [0.96614594 0.96329607 0.99653596 0.98432766 0.99460303]
Средняя оценка ROC-AUC:  0.9809817321142666
CPU times: total: 12min 27s
Wall time: 2min 13s


 #### По результатам кросс-валидации лучшей моделью оказалась `RandomForestClassifier` и `MLPClassifier`, которые показали минимальную разницу между значениями метрик по пяти фолдам на кросс-валидации. Также средняя оценка метрики `Accuracy` выше, чем у других моделей и составляет 92,72% и 92,47% соответственно.

### Оценка моделей на тестовой выбрке

In [80]:
# Обучаем модель RandomForestClassifier и оцениваем на тесте
pipeline2.fit(X_train, y_train)

probabilities_valid1 = pipeline2.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, probabilities_valid1)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.9988646179221338


In [77]:
# Обучаем модель MLPClassifier и оцениваем на тесте
pipeline.fit(X_train, y_train)

probabilities_valid = pipeline.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, probabilities_valid)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.9739734925749939


In [78]:
# Обучаем модель GradientBoostingClassifier и оцениваем на тесте
pipeline1.fit(X_train, y_train)

probabilities_valid = pipeline1.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, probabilities_valid)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.9983788155897664


In [79]:
# Обучаем модель LogisticRegression и оцениваем на тесте
pipeline3.fit(X_train, y_train)

probabilities_valid = pipeline3.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, probabilities_valid)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.9858379081025304


 #### По результатам кросс-валидации и оценки на тесте лучшей моделью оказалась `RandomForestClassifier`, которая показала минимальную разницу между значениями метрики по пяти фолдам на кросс-валидации. Также средняя оценка метрики `ROC-AUC` выше, чем у других моделей и составляет на трейне 99,85% и на тесте 99,88%.

#### Тестирование лучшей модели

In [85]:
print("Classification Report RandomForestClassifier:")
print("Для тренировочной выборки:")
print(classification_report(y_train, pipeline2.predict(X_train)))
print ("Для тестовой выборки:")
print(classification_report(y_test, pipeline2.predict(X_test)))

Classification Report RandomForestClassifier:
Для тренировочной выборки:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     76255
           1       0.96      1.00      0.98       388

    accuracy                           1.00     76643
   macro avg       0.98      1.00      0.99     76643
weighted avg       1.00      1.00      1.00     76643

Для тестовой выборки:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32681
           1       0.99      1.00      0.99       166

    accuracy                           1.00     32847
   macro avg       0.99      1.00      1.00     32847
weighted avg       1.00      1.00      1.00     32847



In [81]:
# Обучение модели на всём датасете
rf_full = pipeline2.fit(features, target)

In [86]:
 # Сохранение обученной модели в файл
with open('rf_full_model.pkl', 'wb') as file:
    pickle.dump(rf_full, file)

In [60]:
# Сохранение OneHotEncoder
with open('onehot_encoder.pkl', 'wb') as f:
    dill.dump(ohe, f)

In [61]:
# Сохранение StandardScaler
with open('scaler.pkl', 'wb') as f:
    dill.dump(std_scaler, f)